# Coroutines

In [ ]:
# Simple example

def simple_coroutine():
    print('croutine started')
    x = yield
    print('coroutine received:', x)

c = simple_coroutine()
print(c)
next(c)


<generator object simple_coroutine at 0x000001FF4FA17B50>
croutine started
coroutine received: 42


StopIteration: 

In [23]:
# Another example

def simple_cor(a):
    print('Started: a =', a)
    b = yield a
    print('Received: b =', b)
    c = yield a + b
    print('Received c =', c)

c = simple_cor(3)
from inspect import getgeneratorstate
print(getgeneratorstate(c))
print(next(c))
getgeneratorstate(c)
print(c.send(2))
c.send(5)


GEN_CREATED
Started: a = 3
3
Received: b = 2
5
Received c = 5


StopIteration: 

### Running average example

In [2]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        num = yield average
        count += 1
        total += num
        average = total / count

### Automatically prime a decorator

In [ ]:
# Example of priming decorator

from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer


## Coroutine Termination and Exception Handling

In [4]:
ca = averager()
next(ca)
ca.send(2)
ca.send('gigi')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [17]:
def demo_exc_handling():
    print('started')
    while True:
        try:
            x = yield
            y = 1 / x
        except TypeError:
            print('Exception handled, continue')
        else:
            print('received: {!r}'.format(x))
    raise RuntimeError('This line should never run')

c = demo_exc_handling()
next(c)
c.send(13)
c.close()

c = demo_exc_handling()
next(c)
c.send(11)
c.send('s')
c.send(0)



started
received: 13
started
received: 11
Exception handled, continue


ZeroDivisionError: division by zero

## Return coroutines results

In [18]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

c = averager()
next(c)
c.send(10)
c.send(2)
c.send(None)

StopIteration: Result(count=2, average=6.0)

In [19]:
c = averager()
next(c)
c.send(2)
c.send(8)

try:
    c.send(None)
except StopIteration as e:
    result = e.value

result


Result(count=2, average=5.0)

## Using yield from

Completely new syntax, could be thinked as await

In [20]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1,3):
        yield i

list(gen())

['A', 'B', 1, 2]

In [22]:
def gen():
    yield from 'AB'
    yield from range(1,3)

list(gen())

['A', 'B', 1, 2]